# Selekcija i evaluacija modela

Selekcija (izbor) optimalnog modela i njegova evaluacija su zadaci koji prate razvoj svakog modela mašinskog učenja. Cilj ove sveske je da približi pojmove i tehnike koje se koriste u praksi. 

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from sklearn import model_selection
from sklearn import svm
from sklearn import metrics
from sklearn import datasets
from sklearn import preprocessing

Skup podataka nad kojim ćemo demonstrirati tehnike selekcije i evaluacije modela je Viskonsin skup podataka za klasifikaciju tumora na benigne i maligne.

In [ ]:
data = datasets.load_breast_cancer()

In [ ]:
X = data.data

In [ ]:
y = data.target

Klasifikator sa kojim ćemo eksperimentisati će biti kernelizovani SVM klasifikator. 

Pod hiperparametrima ili metaparametrima modela podrazumevamo parametre kojima se modeli konfigurišu. Na primer, to mogu biti vrednosti regularizacionih konstanti, broj najbližih suseda $k$ za $k$-nn algoritam, parametri kernela za kernelizovane metode, i mnogi drugi.  

# Selekcija i evaluacija modela koji nemaju parametre

## Skupovi za treniranje i testiranje

U slučejevima kada modeli nemaju hiperparametre koje treba podešavati ili kada ih nije potrebno podešavati (što je u praksi jako retko), možemo podeliti skup podataka na skup za treniranje i skup za testiranje. Važno je upamtiti da ova dva skupa treba da budu **disjunktna**. 

<img src='assets/train_test.png'>

Ovu podelu smo do sada vršili korišćenjem bibliotečke funkcije `train_test_split` pri čemu smo prilikom podele parametrom `test_size` (ili ekvivalentno parametrom `train_size`) naglašavali dimenzije ovih skupova. Takođe, kako je podela skupova bila nasumična, parametrom `random_state` smo uticali na proces ove podele kako bi omogućili da se eksperimenti po potrebi reprodukuju.

Nakon podele dobijene skupove dodatno obrađujemo tako što ih standardizujemo. Srednju vrednost i standardnu devijaciju računamo na osnovu instanci koje se nalaze **isključivo u skupu za treniranje**, i onda tako dobijene vrednosti koristimo i za standardizaciju skupa za testiranje. Do sada smo ovaj zadatak ispunjavali korišćenjem bibliotečke klase `StandardScaler` i njenih metoda `fit` i `transform`.

Model dalje treniramo nad skupom za treniranje i ocenjujemo ga (evaluiramo) na skupu za testiranje birajući neku od metrika relevantnih za zadatak klasifikacije. 

## Unakrsna validacija

Podela skupa podataka na skup za treniranje i skup za testiranje je nasumična tako da možemo imati podelu koja daje optimistično dobre ili nerealno loše rezultate. Zato je praksa da se, kadgod to skupovi podataka i modeli dozvoljavaju, više puta izvrši podela polaznog skupa na skup za treniranje i testiranje tako da svaka od instanci skupa dobije priliku da se nađe i u jednom i u drugom skupu. Jedan takav postupak predstavlja višeslojna unakrsna validacija (engl. cross validation). <img src='assets/cross_validation.png'>

Unakrsna validacija podrazumeva podelu skupa svih podataka na manje grupe, takozvane slojeve (engl. folds), recimo $k$ njih, i treniranje i evaluaciju koji se ponavljaju tačno $k$ puta. U prvom krugu treniranja i evaluacije, prvi sloj se koristi za testiranje, a preostalih $k-1$ slojeva se koristi za treniranje modela. U drugom krugu treniranja i evaluacije, drugi sloj se koristi sa testiranje, a prestalih $k-1$ slojeva za treniranje. U poslednjem krugu treniranja i evaluacije, poslednji sloj se koristi za testiranje, a prvih $k-1$ slojeva za treniranje modela. 

Na slici je prikazana unakrsna validacija sa 7 slojeva. U praksi se najčešće koriste podele na 3, 5 ili 10 slojeva. Posebno, ukoliko broj slojeva odgovara broju instanci u skupu, možemo govoriti o `leave-one-out` tehnici. Nju smo imali prilike da vidimo u primeru vezanom za Nadaraja-Votson regresiju. 

Biblioteka `scikit-learn` preko klase `KFold` omogućava particionisanje skupa podataka na opisani način. Nakon što se zada broj slojeva preko parametra `n_splits` metodom `split` se za svaku iteraciju generiše niz skupova indeksa instanci koje treba da se nađu u skupu za treniranje i skupu za testiranje. 

Sledeći primer ilustruje proces za unakrsnu validaciju sa 10 slojeva.

### Bibliotečka podrška unakrsnoj validaciji

Pošto je zadatak unakrsne validacije čest, biblioteka `scikit-learn` stavlja na raspolaganje funkciju `cross_val_score` koja vrši unakrsnu validaciju. Ova funkcija od parametara očekuje model koji se evaluira, ceo skup podataka, funkciju za ocenu modela i broj slojeva (ili podeone indekse koji se u njima nalaze).

In [ ]:
X = data.data

In [ ]:
y = data.target

Možemo primetiti da su vrednosti koje dobijamo nešto drugačije (niže) od vrednosti koje smo dobili prilikom prethodne implementacije. Ovo ponašanje dolazi kao posledica treniranja i testiranja modela na odgovarajućim podelama skupova koje nisu standardizovane. Njega možemo popraviti uvođenjem takozvanih `cevovoda` (engl. pipeline) koji definišu niz koraka koji treba sprovesti. Podrška za rad sa cevovodima dolazi kroz `pipeline` paket `scikit-learn` biblioteke.

In [ ]:
from sklearn import pipeline

## Finalni model

Finalni model je model koji obučavamo nad celim skupom podataka.

In [ ]:
X = data.data
y = data.target

In [ ]:
final_scaler = preprocessing.StandardScaler()
X = final_scaler.fit_transform(X)

In [ ]:
final_model = svm.SVC(X, y)

U praksi ovaj korak često nije realističan, pogotovo kada je reč o kompleksnim modelima čije treniranje dugo traje ili zahteva veliku količinu resursa. Zato je često scenario da se za finalni model uzima model koji je obučen na skupu za treniranje. 

# Selekcija i evaluacija modela sa parametrima

## Skupovi za treniranje, validaciju i testiranje

Kako bi se odredile optimalne vrednosti parametara modela, praksa je da se polazni skup podataka podeli na skup za treniranje, skup za validaciju i skup za testiranje. Model se trenira nad skupom za treniranje za različite vrednosti parametara i ocenjuje na skupu za validaciju. Na osnovu ovako dobijenih rezultata se biraju optimalna vrednosti parametara. Dalje se ovaj model evaluira na skupu za testiranje i tako dobijena ocena se smatra ocenom modela. 
<img src='assets/train_validation_test.png'>

Opisanu podelu skupa podataka možemo dobiti bibliotečkom funkcijom `train_test_split`.  

In [ ]:
X = data.data
y = data.target

### Unakrsna validacija

Podela skupa podataka na skup za treniranje, validaciju i testiranje ima iste nedostatke kao i podela opisana u slučaju neparametarskih modela. Stoga se može koristiti unakrsna validacija. 

#### Unakrsna validacija za evaluaciju modela sa parametrima

Ukoliko su vrednosti hiperparameta poznate, unakrsna validacija se primenjuje na način koji smo do sada upoznali.

In [ ]:
X = data.data
y = data.target

#### Određivanje vrednosti hiperparametra unakrsnom validacijom

Unakrsna validacija se može koristiti i za određivanje vrednosti hiperparametara modela. Ovde treba obratiti pažnju da se prosleđuje unija trening i validacinog skupa metodi `cross_val_score`.

In [ ]:
Cs = np.array([10**i for i in range(-5, 5)])
gammas = np.array([10**i for i in range(-3, 3)])

Biblioteka `scikit-learn` nudi podršku za korišćenje unakrsne validacije za određivanje vrednosti hiperparametara pomoću klase `GridSearchCV`(CV na kraju naziva metode je akronim za `cross-validation`). Konstruktor očekuje model za koji treba odrediti vrednosti parametara, rečnik svih parametara koje treba razmotriti, funkciju za ocenu modela, broj slojeva i, opciono, parametar kojim se naglašava da se svi dobijeni rezultati mogu sačuvati u formi izveštaja. Potom se metodom `fit` vrši pretraga prostora parametara na gore opisani način, takođe nad unijom skupa za treniranje i validaciju.

Informacije o svakom koraku unakrsne validacije za svaku kombinaciju vrednosti hiperparametara su dostupne kroz izveštaj `cv_results_`. Broj različitih vrednosti parametra `C` je 10, a parametra `gamma` 6. 

Informacije iz izveštaja se mogu koristiti i za uvid u opsege vrednosti iz kojih su hiperparametri birani. To se obično radi vizuelizacijom u vidu toplotne mape. Ukoliko su opsezi vrednosti dobro birani, optimalne vrednosti su skoncentrisane po sredini.

In [ ]:
cv_test_score = cv_results['mean_test_score'].values.reshape(6, 10)

Cs = [10**i for i in range(-5, 5)]
gammas =[10**i for i in range(-3, 3)]

plt.figure(figsize=(10, 8))

plt.ylabel('gamma')
plt.yticks(np.arange(0, len(gammas)), gammas)

plt.xlabel('C')
plt.xticks(np.arange(0, len(Cs)), Cs)

plt.imshow(cv_test_score, cmap='viridis')
plt.colorbar()
plt.show()

U našem slučaju bi imalo smisla ispitati ponašanje modela i za vrednosti parametra `gamma` koje su nešto veće od 100.

## Ugnježđena unakrsna validacija

Korišćenje unakrsne validacije za određivanje optimalne vrednosti hiperparametara i dalje ostavlja otvorenim podelu skupa podataka na skup za treniranje i validaciju sa jedne strane, i skup za testiranje sa druge strane. I za drugo se može koristiti, baš kao i u slučaju neparametarskih modela, unakrsna validacija. Time dobijamo ugnježđenu unakrsnu validaciju (engl. nested cross-validation). <img src='assets/nested_cross_validation.png'>

Ona se na nivou biblioteke jednostavno implementira kombinacijom `GridSearchCV` i `cross_val_score` funkcija.

## Finalni model

Finalni model dobijamo kada za vrednosti parametara uzmemo optimalne vrednosti dobijene ili korišćenjem validacionog skupa ili korišćenjem unakrsne validacije i istreniramo ga na celom skupu podataka.

In [ ]:
X = data.data
y = data.target

In [ ]:
final_scaler = preprocessing.StandardScaler()
X = final_scaler.fit_transform(X)

In [ ]:
final_model = svm.SVC(C=best_params['C'], gamma=best_params['gamma'])

In [ ]:
final_model.fit(X, y)

Baš kao i kod neparametarskih modela u praksi se finalni parametarski modeli koji zahtevaju velike resurse ili čije treniranje dugo traje ne treniraju na celom skupu, već se za finalni model prograšava model dobijen na skupu za treniranje ili skupu za treniranje i validaciju. 